In [1]:
import pandas as pd

In [3]:
!dir

 Volume in drive C is OSDisk
 Volume Serial Number is 4836-71B9

 Directory of C:\Users\rahulgup\Downloads\Kaggle_codes

04/30/2020  07:29 PM    <DIR>          .
04/30/2020  07:29 PM    <DIR>          ..
04/30/2020  07:29 PM    <DIR>          .ipynb_checkpoints
04/30/2020  03:18 PM           216,309 covid19-global-forecasting-week-2.zip
04/30/2020  07:28 PM           115,351 submission.csv
04/30/2020  07:28 PM           365,792 test.csv
04/30/2020  07:28 PM           783,368 train.csv
04/30/2020  07:29 PM                72 Untitled.ipynb
               5 File(s)      1,480,892 bytes
               3 Dir(s)  138,435,411,968 bytes free


In [89]:
train = pd.read_csv("train.csv", keep_default_na=False)

In [9]:
test = pd.read_csv("test.csv", keep_default_na=False)

In [10]:
submission = pd.read_csv("submission.csv", keep_default_na=False)

In [16]:
print(
    train.shape, test.shape, submission.shape
)

((20580, 6), (12642, 4), (12642, 3))


In [90]:
train.dtypes

Id                  int64
Province_State     object
Country_Region     object
Date               object
ConfirmedCases    float64
Fatalities        float64
dtype: object

In [14]:
train.head(10)

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,,Afghanistan,2020-01-22,0.0,0.0
1,2,,Afghanistan,2020-01-23,0.0,0.0
2,3,,Afghanistan,2020-01-24,0.0,0.0
3,4,,Afghanistan,2020-01-25,0.0,0.0
4,5,,Afghanistan,2020-01-26,0.0,0.0
5,6,,Afghanistan,2020-01-27,0.0,0.0
6,7,,Afghanistan,2020-01-28,0.0,0.0
7,8,,Afghanistan,2020-01-29,0.0,0.0
8,9,,Afghanistan,2020-01-30,0.0,0.0
9,10,,Afghanistan,2020-01-31,0.0,0.0


In [13]:
test.head(10)

,ForecastId,Province_State,Country_Region,Date
0,1,,Afghanistan,2020-03-19
1,2,,Afghanistan,2020-03-20
2,3,,Afghanistan,2020-03-21
3,4,,Afghanistan,2020-03-22
4,5,,Afghanistan,2020-03-23
5,6,,Afghanistan,2020-03-24
6,7,,Afghanistan,2020-03-25
7,8,,Afghanistan,2020-03-26
8,9,,Afghanistan,2020-03-27
9,10,,Afghanistan,2020-03-28


In [15]:
submission.head(10)

,ForecastId,ConfirmedCases,Fatalities
0,1,1,1
1,2,1,1
2,3,1,1
3,4,1,1
4,5,1,1
5,6,1,1
6,7,1,1
7,8,1,1
8,9,1,1
9,10,1,1


### K-Means clustering method considers two assumptions regarding the clusters – first that the clusters are spherical and second that the clusters are of similar size.
- Spherical assumption helps in separating the clusters when the algorithm works on the data and forms clusters. If this assumption is violated, the clusters formed may not be what one expects.
- On the other hand, assumption over the size of clusters helps in deciding the boundaries of the cluster. This assumption helps in calculating the number of data points each cluster should have.
- This assumption also gives an advantage. Clusters in K-means are defined by taking the mean of all the data points in the cluster. With this assumption, one can start with the centers of clusters anywhere.
- Keeping the starting points of the clusters anywhere will still make the algorithm converge with the same final clusters as keeping the centers as far apart as possible.

In [27]:
train.shape, len(train.Id.unique())

((20580, 6), 20580)

In [28]:
train.shape, len(train.Province_State.unique())

((20580, 6), 126)

In [29]:
train.shape, len(train.Country_Region.unique())

((20580, 6), 173)

In [30]:
train.shape, len(train.Date.unique())

((20580, 6), 70)

In [31]:
train.shape, len(train.ConfirmedCases.unique())

((20580, 6), 1536)

In [32]:
train.shape, len(train.Fatalities.unique())

((20580, 6), 328)

In [40]:
train.describe()

,Id,ConfirmedCases,Fatalities
count,20580.000000,20580.000000,20580.000000
mean,14685.500000,514.939116,21.080952
std,8487.230117,4541.261768,287.758197
min,1.000000,0.000000,0.000000
25%,7335.750000,0.000000,0.000000
50%,14685.500000,0.000000,0.000000
75%,22035.250000,35.000000,0.000000
max,29370.000000,105792.000000,12428.000000


In [63]:
df = train.groupby(train.Province_State).nunique()
df[df.Province_State > 1]

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
Province_State,,,,,,


In [91]:
df = train.groupby(train.Country_Region).nunique()
df[df.Province_State > 1]

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
Country_Region,,,,,,
Australia,560,8,1,70,112,9
Canada,700,10,1,70,120,20
China,2310,33,1,70,657,83
Denmark,210,3,1,70,51,17
France,700,10,1,70,87,27
Netherlands,280,4,1,70,47,24
US,3780,54,1,70,540,108
United Kingdom,490,7,1,70,67,23


In [102]:
df[df.Province_State > 1].index

Index([u'Australia', u'Canada', u'China', u'Denmark', u'France',
       u'Netherlands', u'US', u'United Kingdom'],
      dtype='object', name=u'Country_Region')

In [107]:
# train[train.Country_Region == "US"][train.Province_State == ""]
for i in df[df.Province_State > 1].index:
    z = [x for x in train[train.Country_Region == i].Province_State.unique() if x == ""]
    k = []
    if z:
        k = train[train.Country_Region == i].Province_State.unique()
    print(i, z, k)

('Australia', [], [])
('Canada', [], [])
('China', [], [])
('Denmark', [''], array(['Faroe Islands', 'Greenland', ''], dtype=object))
('France', [''], array(['French Guiana', 'French Polynesia', 'Guadeloupe', 'Martinique',
       'Mayotte', 'New Caledonia', 'Reunion', 'Saint Barthelemy',
       'St Martin', ''], dtype=object))
('Netherlands', [''], array(['Aruba', 'Curacao', 'Sint Maarten', ''], dtype=object))
('US', [], [])
('United Kingdom', [''], array(['Bermuda', 'Cayman Islands', 'Channel Islands', 'Gibraltar',
       'Isle of Man', 'Montserrat', ''], dtype=object))


In [93]:
# df["US" in df.Country_Region]#.Province_State.unique()

In [69]:
df = train.groupby([train.Province_State, train.Country_Region]).nunique()
# df = train.groupby(train.Country_Region).nunique()
df[df.Province_State == 1]

,,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
Province_State,Country_Region,,,,,,


In [108]:
train['Province_State'] = train.apply(
    lambda row: row['Country_Region'] if row['Province_State'] == "" else row['Province_State'],
    axis=1
)

In [109]:
# train.groupby([train.Province_State, train.Country_Region]).nunique()

In [110]:
len(train.Province_State.unique()), len(train.Country_Region.unique())

(293, 173)

In [64]:
train.Date.unique()

array(['2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25',
       '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29',
       '2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02',
       '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06',
       '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10',
       '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14',
       '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18',
       '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22',
       '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26',
       '2020-02-27', '2020-02-28', '2020-02-29', '2020-03-01',
       '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05',
       '2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09',
       '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13',
       '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17',
       '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-21',
       '2020-03-22', '2020-03-23', '2020-03-24', '2020-

In [115]:
[x for x in train.Fatalities.unique() if x == float("NaN")]

[]

In [116]:
[x for x in train.ConfirmedCases.unique() if x == float("NaN")]

[]

In [138]:
df2 = train.groupby(train.Country_Region).max()
df2["country"] = df2.index.values

In [139]:
df2 = df2[["country", "ConfirmedCases", "Fatalities"]]

In [140]:
df2["case_rank"] = df2["ConfirmedCases"].rank()
df2["fat_rank"] = df2["Fatalities"].rank()

In [141]:
df2

,country,ConfirmedCases,Fatalities,case_rank,fat_rank
Country_Region,,,,,
Afghanistan,Afghanistan,174.0,4.0,81.0,90.5
Albania,Albania,243.0,15.0,89.0,122.5
Algeria,Algeria,716.0,44.0,119.0,145.0
Andorra,Andorra,376.0,12.0,102.0,118.0
Angola,Angola,7.0,2.0,17.5,76.0
Antigua and Barbuda,Antigua and Barbuda,7.0,0.0,17.5,25.5
Argentina,Argentina,1054.0,27.0,127.0,134.0
Armenia,Armenia,532.0,3.0,109.0,85.0
Australia,Australia,2032.0,8.0,142.0,109.0


In [142]:
df2.sort_values(by=["case_rank"])

,country,ConfirmedCases,Fatalities,case_rank,fat_rank
Country_Region,,,,,
Timor-Leste,Timor-Leste,1.0,0.0,2.0,25.5
Papua New Guinea,Papua New Guinea,1.0,0.0,2.0,25.5
Saint Vincent and the Grenadines,Saint Vincent and the Grenadines,1.0,0.0,2.0,25.5
Liberia,Liberia,3.0,0.0,5.0,25.5
Central African Republic,Central African Republic,3.0,0.0,5.0,25.5
Belize,Belize,3.0,0.0,5.0,25.5
Gambia,Gambia,4.0,1.0,7.5,60.5
Bhutan,Bhutan,4.0,0.0,7.5,25.5
Nicaragua,Nicaragua,5.0,1.0,10.5,60.5


In [143]:
df2.sort_values(by=["fat_rank"])

,country,ConfirmedCases,Fatalities,case_rank,fat_rank
Country_Region,,,,,
Central African Republic,Central African Republic,3.0,0.0,5.0,25.5
Chad,Chad,7.0,0.0,17.5,25.5
Mongolia,Mongolia,12.0,0.0,34.0,25.5
Congo (Brazzaville),Congo (Brazzaville),19.0,0.0,42.5,25.5
Malta,Malta,169.0,0.0,79.0,25.5
Maldives,Maldives,18.0,0.0,41.0,25.5
Madagascar,Madagascar,57.0,0.0,59.0,25.5
Liechtenstein,Liechtenstein,68.0,0.0,62.0,25.5
Libya,Libya,10.0,0.0,30.0,25.5
